In [19]:
import time
start_time = time.time()

#Where is the data?
if username == "bdutt":
    baseDir="/Users/bdutt/Documents/workspace-whitespace/face/face/data/lfw"
elif username =="cgarg":
    baseDir="/Users/cgarg/Documents/"
flow = "main"
debug = False

%run "Load Data.ipynb"

filenames = []
labels = []

def collectFileNamesAndLabels(filename):
    filenames.append(filename)
    labels.append("Fall" in filename) #Get the label name from filename
        

doForEachFile(collectFileNamesAndLabels,baseDir)
print("Total File name collection time = %s seconds" % (time.time() - start_time))
print ("Files collected : ",len(filenames))

Total File name collection time = 1.3245849609375 seconds
Files collected :  13233


In [20]:
imagesList = []
doForEachFileNames(readImage, filenames, imagesList)
print ("Images data collected : ",len(imagesList))

Images data collected :  13233


In [ ]:
clf = OmClassifier()
params = {
    "epoch" : 500,
    "trace" : 1,
    "rnnVerbose" : 2
}
clf.set_params(**params)
clf.fit(np.array(filenames).reshape(-1, 1), labels)